# Construindo a Arquitetura da CNN 

![Extracao de características](extracao_caracteristicas.png)

## 1. Importando as bibliotecas

In [6]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras import utils
import numpy as np

## 2. Aquisição dos dados

In [9]:
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

In [10]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


## 3. Pré-processamento

In [11]:
X_train = X_train / 255.
X_test = X_test / 255.
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)
y_train = utils.to_categorical(y_train) #8 -> 0 0 0 0 0 0 0 0 1 0
y_test = utils.to_categorical(y_test) #3 -> 0 0 0 1 0 0 0 0 0 0

In [12]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(60000, 28, 28, 1)
(60000, 10)
(10000, 28, 28, 1)
(10000, 10)


## 4. Arquitetura da CNN

![Arquitetura CNN](cnn_arquitetura_basica.png)

In [9]:
# Inicializando a CNN
classifier = Sequential()

#Camada de convolução
classifier.add(Convolution2D(32, kernel_size=(3,3), input_shape = (28, 28,1), activation = 'relu', padding='same', name = 'conv_1'))

#Camada de pooling
classifier.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2), padding='same', name = 'pool_1'))

#Segunda camada convolucional
classifier.add(Convolution2D(64, kernel_size=(3,3), activation = 'relu', padding='same', name = 'conv_2'))

#Dropout
classifier.add(Dropout(0.25))

#Segunda camada de pooling
classifier.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same', name = 'pool_2'))


#Vetorizando os mapas de características do último pooling (camada de entrada)
classifier.add(Flatten())

#Dropout
classifier.add(Dropout(0.5))

#Camada totalmente conectada ou oculta
classifier.add(Dense(activation='relu', units=128, name = 'dense_1'))


#Camada de saída
classifier.add(Dense(activation='softmax', units=10,  name = 'classification'))

$Atividade (2.5):$  Classificando um conjunto de imagens de animais (gato, cachorro e panda)
1. Aquisição das imagens através do dataset: https://www.kaggle.com/ashishsaxena2209/animal-image-datasetdog-cat-and-panda
2. Pré-processamento: Redimensione a imagem do dataset para 64 × 64 pixels, e certifique-se de que tenham os três canais (RGB). As imagens nas quais o modelo deve ser treinado devem ser pré-processadas para que seus valores variem de [0,1], portanto, você deve garantir que suas imagens também estejam no mesmo intervalo. As labels devem ser criadas e transformadas em dados categóricos e não esqueçam de embaralhar as amostras de modo que a label também seja embaralhada na mesma sequência. Dica: função load_img do keras.preprocessing.image e utilizar 80% das amostras para treino e 20% para teste;
3. Adaptar a arquitetura apresentada no tutorial para o novo dataset. 

In [10]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_1 (Conv2D)              (None, 28, 28, 32)        320       
_________________________________________________________________
pool_1 (MaxPooling2D)        (None, 14, 14, 32)        0         
_________________________________________________________________
conv_2 (Conv2D)              (None, 14, 14, 64)        18496     
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 64)        0         
_________________________________________________________________
pool_2 (MaxPooling2D)        (None, 7, 7, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 3136)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 3136)              0

## 5. Treinando o modelo

In [17]:
#Parâmetros de treinamento
epochs = 60
batch_size = 50
validation_split=0.1

In [4]:
print(54000/50)

1080.0


In [21]:
classifier.compile(optimizer = 'sgd', loss= 'categorical_crossentropy', metrics=['accuracy'])

checkpoint = keras.callbacks.ModelCheckpoint('best_model.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='auto', save_freq='epoch') 
earlystop = keras.callbacks.EarlyStopping(patience=15)

In [22]:
classifier.fit(X_train, y_train, validation_split=validation_split, batch_size=batch_size, epochs=epochs, callbacks=[checkpoint,earlystop], verbose=1)

Epoch 1/60
1079/1080 [============================>.] - ETA: 0s - loss: 0.7852 - accuracy: 0.7497
Epoch 00001: val_loss improved from inf to 0.25069, saving model to best_model.h5
1080/1080 [==============================] - 29s 27ms/step - loss: 0.7848 - accuracy: 0.7498 - val_loss: 0.2507 - val_accuracy: 0.9402
Epoch 2/60
1079/1080 [============================>.] - ETA: 0s - loss: 0.2706 - accuracy: 0.9170
Epoch 00002: val_loss improved from 0.25069 to 0.13896, saving model to best_model.h5
1080/1080 [==============================] - 26s 24ms/step - loss: 0.2705 - accuracy: 0.9170 - val_loss: 0.1390 - val_accuracy: 0.9663
Epoch 3/60
1078/1080 [============================>.] - ETA: 0s - loss: 0.1905 - accuracy: 0.9427
Epoch 00003: val_loss improved from 0.13896 to 0.11194, saving model to best_model.h5
1080/1080 [==============================] - 28s 26ms/step - loss: 0.1904 - accuracy: 0.9427 - val_loss: 0.1119 - val_accuracy: 0.9718
Epoch 4/60
1080/1080 [=========================

## 6. Avaliando o modelo

In [7]:
best_model = keras.models.load_model("best_model.h5")

In [13]:
score = best_model.evaluate(X_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.023409845307469368
Test accuracy: 0.9916999936103821


𝐴𝑡𝑖𝑣𝑖𝑑𝑎𝑑𝑒(2.5):  Treinar e Avaliar a arquitetura adaptada para o conjunto de imagens de animais (gato, cachorro e panda)

1. Modifiquem os hiperparâmetros: epochs, batch_size e optimizer a fim de melhorar a acurácia de validação. Quando alcançar um conjunto, avaliar o desempenho de classificação do melhor modelo com os dados de teste;
2. Modificar a arquitetura: inserir e remover camadas, alterar profundidade das camadas e comparar os resultados de classificação no conjunto de teste com aquele alcançado no item 1. 